In [1]:
import sys
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
sys.path.append(".")
# os.environ["MAX_JOBS"] = "100"

import torch
torch.set_printoptions(linewidth=200)
torch.cuda.is_available()

True

In [2]:
from src.qkv_v1.interface import qkvkernel

INCLUDE: ['/home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include', '/home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/torch/csrc/api/include', '/home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/TH', '/home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/THC', '/home/max/miniconda3/envs/xlstmpt21cu118/include']
/home/max/myrepos/vlstm_cuda/src
/home/max/cpplibs/libtorch/lib:/usr/local/cuda-12.3/lib64:
/home/max/miniconda3/envs/xlstmpt21cu118/lib


Using /home/max/.cache/torch_extensions/py311_cu118 as PyTorch extensions root...
Creating extension directory /home/max/.cache/torch_extensions/py311_cu118/qkv_v1...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/max/.cache/torch_extensions/py311_cu118/qkv_v1/build.ninja...
Building extension module qkv_v1...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/3] /home/max/miniconda3/envs/xlstmpt21cu118/bin/x86_64-conda-linux-gnu-c++ -MMD -MF interface.o.d -DTORCH_EXTENSION_NAME=qkv_v1 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include -isystem /home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/torch/csrc/api/include -isystem /home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/TH -isystem /home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/THC -isystem /home/max/miniconda3/envs/xlstmpt21cu118/include -isystem /home/max/miniconda3/envs/xlstmpt21cu118/include/python3.11 -D_GLIBCXX_USE_CXX11_ABI=0 -fPIC -std=c++17 -c /home/max/myrepos/vlstm_cuda/src/qkv_v1/interface.cc -o interface.o 
[2/3] /home/max/miniconda3/envs/xlstmpt21cu118/bin/nvcc  -ccbin /home/max/miniconda3/env

Loading extension module qkv_v1...


### qkvtest

In [3]:
S = 16 # seq len
B = 1 # batch size
NH = 1 # num heads
DH = 4 # dim per head
DTYPE = torch.float32
DEVICE = torch.device("cuda:0")

In [4]:
# create qkv, inputgates, forgetgates 
torch.manual_seed(0)
qs = torch.arange((B*NH*S*DH), device=DEVICE, dtype=DTYPE).reshape((B, NH, S, DH)) / 10.
ks = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) #/ 10.
vs = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) #/ 10.
qs, qs.shape

(tensor([[[[0.0000, 0.1000, 0.2000, 0.3000],
           [0.4000, 0.5000, 0.6000, 0.7000],
           [0.8000, 0.9000, 1.0000, 1.1000],
           [1.2000, 1.3000, 1.4000, 1.5000],
           [1.6000, 1.7000, 1.8000, 1.9000],
           [2.0000, 2.1000, 2.2000, 2.3000],
           [2.4000, 2.5000, 2.6000, 2.7000],
           [2.8000, 2.9000, 3.0000, 3.1000],
           [3.2000, 3.3000, 3.4000, 3.5000],
           [3.6000, 3.7000, 3.8000, 3.9000],
           [4.0000, 4.1000, 4.2000, 4.3000],
           [4.4000, 4.5000, 4.6000, 4.7000],
           [4.8000, 4.9000, 5.0000, 5.1000],
           [5.2000, 5.3000, 5.4000, 5.5000],
           [5.6000, 5.7000, 5.8000, 5.9000],
           [6.0000, 6.1000, 6.2000, 6.3000]]]], device='cuda:0'),
 torch.Size([1, 1, 16, 4]))

In [29]:
len(qs.view(-1))

64

In [30]:
# rs = qs @ ks.transpose(-1, -2)
# rs, rs.shape

In [5]:
# pytorch version
rs = qs @ ks.transpose(-1, -2) @ vs
rs, rs.shape

(tensor([[[[  9.6000,   9.6000,   9.6000,   9.6000],
           [ 35.2000,  35.2000,  35.2000,  35.2000],
           [ 60.8000,  60.8000,  60.8000,  60.8000],
           [ 86.4000,  86.4000,  86.4000,  86.4000],
           [112.0000, 112.0000, 112.0000, 112.0000],
           [137.6000, 137.6000, 137.6000, 137.6000],
           [163.2000, 163.2000, 163.2000, 163.2000],
           [188.8000, 188.8000, 188.8000, 188.8000],
           [214.4000, 214.4000, 214.4000, 214.4000],
           [240.0000, 240.0000, 240.0000, 240.0000],
           [265.6000, 265.6000, 265.6000, 265.6000],
           [291.2000, 291.2000, 291.2000, 291.2000],
           [316.8000, 316.8000, 316.8000, 316.8000],
           [342.4000, 342.4000, 342.4000, 342.4000],
           [368.0000, 368.0000, 368.0000, 368.0000],
           [393.6000, 393.6000, 393.6000, 393.6000]]]], device='cuda:0'),
 torch.Size([1, 1, 16, 4]))

In [6]:
# cuda kernel
rs = qkvkernel(mat_Q=qs, mat_K=ks, mat_V=vs)
rs, rs.shape

before kernel dispatch - float32!
B: 1, NH: 1, S: 16, DH: 4
blocksxy: 1-2, threads: 4-4, shared_mem in bytes: 2048
In Kernel: gdim.x: 1, gdim.y: 2, gdim.z: 1, bdim.x: 4, bdim.y: 4
In Kernel: QtileDim: 8, KVtileDim: 8, TblockDim:4


(tensor([[[[  9.6000,   9.6000,   9.6000,   9.6000],
           [ 35.2000,  35.2000,  35.2000,  35.2000],
           [ 60.8000,  60.8000,  60.8000,  60.8000],
           [ 86.4000,  86.4000,  86.4000,  86.4000],
           [112.0000, 112.0000, 112.0000, 112.0000],
           [137.6000, 137.6000, 137.6000, 137.6000],
           [163.2000, 163.2000, 163.2000, 163.2000],
           [188.8000, 188.8000, 188.8000, 188.8000],
           [214.4000, 214.4000, 214.4000, 214.4000],
           [240.0000, 240.0000, 240.0000, 240.0000],
           [265.6001, 265.6001, 265.6001, 265.6001],
           [291.2000, 291.2000, 291.2000, 291.2000],
           [316.8000, 316.8000, 316.8000, 316.8000],
           [342.4000, 342.4000, 342.4000, 342.4000],
           [368.0000, 368.0000, 368.0000, 368.0000],
           [393.6001, 393.6001, 393.6001, 393.6001]]]], device='cuda:0'),
 torch.Size([1, 1, 16, 4]))

### Matmul kernel Test from vlstm_v2

In [8]:
# from src.vlstm_v3.interface import testkernel, copykernel, mmkernelv1

In [7]:
DTYPE = torch.bfloat16
DEVICE = torch.device("cuda:0")

In [10]:
S = 8 # sequence length
DH = 8 # hidden size

In [11]:
matA = torch.arange((2*S * DH), device=DEVICE, dtype=DTYPE).reshape((2*S, DH))
matB = torch.ones((DH, S), device=DEVICE, dtype=DTYPE)
matA.shape, matB.shape, matA, matA.sum(-1)

(torch.Size([16, 8]),
 torch.Size([8, 8]),
 tensor([[  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.],
         [  8.,   9.,  10.,  11.,  12.,  13.,  14.,  15.],
         [ 16.,  17.,  18.,  19.,  20.,  21.,  22.,  23.],
         [ 24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.],
         [ 32.,  33.,  34.,  35.,  36.,  37.,  38.,  39.],
         [ 40.,  41.,  42.,  43.,  44.,  45.,  46.,  47.],
         [ 48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.],
         [ 56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.],
         [ 64.,  65.,  66.,  67.,  68.,  69.,  70.,  71.],
         [ 72.,  73.,  74.,  75.,  76.,  77.,  78.,  79.],
         [ 80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.],
         [ 88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.],
         [ 96.,  97.,  98.,  99., 100., 101., 102., 103.],
         [104., 105., 106., 107., 108., 109., 110., 111.],
         [112., 113., 114., 115., 116., 117., 118., 119.],
         [120., 121., 122., 123., 124., 125., 126., 127.]], device='cuda

In [12]:
# pytorch
pt_out = matA @ matB
pt_out, pt_out.shape

(tensor([[ 28.,  28.,  28.,  28.,  28.,  28.,  28.,  28.],
         [ 92.,  92.,  92.,  92.,  92.,  92.,  92.,  92.],
         [156., 156., 156., 156., 156., 156., 156., 156.],
         [220., 220., 220., 220., 220., 220., 220., 220.],
         [284., 284., 284., 284., 284., 284., 284., 284.],
         [348., 348., 348., 348., 348., 348., 348., 348.],
         [412., 412., 412., 412., 412., 412., 412., 412.],
         [476., 476., 476., 476., 476., 476., 476., 476.],
         [540., 540., 540., 540., 540., 540., 540., 540.],
         [604., 604., 604., 604., 604., 604., 604., 604.],
         [668., 668., 668., 668., 668., 668., 668., 668.],
         [732., 732., 732., 732., 732., 732., 732., 732.],
         [796., 796., 796., 796., 796., 796., 796., 796.],
         [860., 860., 860., 860., 860., 860., 860., 860.],
         [924., 924., 924., 924., 924., 924., 924., 924.],
         [988., 988., 988., 988., 988., 988., 988., 988.]], device='cuda:0', dtype=torch.bfloat16),
 torch.Size([16

In [13]:
matA.is_contiguous(), matB.is_contiguous()

(True, True)

In [14]:
cu_out = mmkernelv1(mat_A=matA, mat_B=matB)
cu_out, cu_out.shape

NameError: name 'mmkernelv1' is not defined

In [ ]:
matA[9]

In [ ]:
matA[9].cumsum(-1)

In [ ]:
torch.arange(72,80, dtype=torch.bfloat16, device=torch.device('cuda:0')).cumsum(-1)

In [ ]:
torch.arange(72,80, dtype=torch.bfloat16, device=torch.device('cpu')).cumsum(-1)

In [ ]:
torch.arange(72,80, dtype=torch.float32, device=torch.device('cpu')).cumsum(-1)

In [ ]:
torch.arange(72,80, dtype=torch.float16, device=torch.device('cuda:0')).cumsum(-1)

In [8]:
# cu_out = mmkernelv2(mat_A=matA, mat_B=matB)
# cu_out, cu_out.shape

In [ ]:
# mat @ mat.T @ mat

### pytorch version

In [ ]:
H = 6 # hidden size
S = 5 # seq len
B = 1 # batch size
NH = 2 # num heads
DH = H // NH # dim per head
DTYPE = torch.bfloat16
DEVICE = torch.device("cuda:0")
assert H % NH == 0

In [ ]:
# create qkv, inputgates, forgetgates 
torch.manual_seed(0)
qs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
ks = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
vs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
ds = torch.rand((B, NH, S, S), device=DEVICE, dtype=DTYPE)

max_log_D, _ = torch.max(ds.view(B, NH, -1), dim=-1, keepdim=True)  # (B, NH, 1)
log_D_matrix_stabilized = ds - max_log_D.unsqueeze(-1)  # (B, NH, S, S) = (B, NH, S, S) - (B, NH, 1, 1)
D_matrix = torch.exp(log_D_matrix_stabilized)  # (B, NH, S, S)
mval = torch.exp(-max_log_D.unsqueeze(-1))